<a href="https://colab.research.google.com/github/obeabi/DeepLearning/blob/master/DogsVsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Written by Abiola Obembe
## The Asirra (Dogs VS Cats) dataset
### Date 2020-09-19
#### Summary: The Asirra (animal species image recognition for restricting access) dataset was introduced in 2013 for a machine learning competition. The dataset includes 25,000 images with equal numbers of labels for cats and dogs.